In [10]:

import os, time
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import ListSortOrder

load_dotenv = True
if load_dotenv:
    from dotenv import load_dotenv
    load_dotenv()

In [12]:
endpoint = os.getenv("PROJECT_ENDPOINT")
print(f"Using endpoint: {endpoint}")
credential = DefaultAzureCredential()

agents_client = AgentsClient(endpoint=endpoint, credential=credential)

Using endpoint: https://aq-ai-foundry-sweden-central.services.ai.azure.com/api/projects/firstProject


In [13]:
# [START create_agent]

agent = agents_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="my-agent",
    instructions="You are helpful agent",
)
print(f"Created agent, agent ID: {agent.id}")

Created agent, agent ID: asst_i70VpISDZ0nTozgVeNcrZJiy


In [14]:
    # [START create_thread]
    thread = agents_client.threads.create()
    # [END create_thread]
    print(f"Created thread, thread ID: {thread.id}")

Created thread, thread ID: thread_VoYokYCsevjIrLUegbfSBija


In [16]:
    # [START create_message]
    message = agents_client.messages.create(thread_id=thread.id, role="user", content="Hello, tell me a joke")
    # [END create_message]
    print(f"Created message, message ID: {message.id}")

Created message, message ID: msg_7QolAjpsaap6y8h6tRk2ldVT


In [17]:
    # [START create_run]
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id)


In [20]:
    # Poll the run as long as run status is queued or in progress
    while run.status in ["queued", "in_progress", "requires_action"]:
        # Wait for a second
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
        # [END create_run]
        print(f"Run status: {run.status}")

        if run.status == "failed":
            print(f"Run error: {run.last_error}")

In [21]:
    agents_client.delete_agent(agent.id)
    print("Deleted agent")

Deleted agent


In [22]:
    # [START list_messages]
    messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role}: {last_text.text.value}")
    # [END list_messages]

MessageRole.USER: Hello, tell me a joke
MessageRole.AGENT: Sure! Why don’t skeletons fight each other?

They don’t have the guts.
